# Day 4: Agents and Tools

This is a continuation of my AI Agents Crash Course.  

In the first three days, I:
- Downloaded data from my repo (Day 1)  
- Processed it into chunks (Day 2)  
- Indexed it with hybrid search (Day 3)  

Now for Day 4, I’m making things more **agentic**:  
- I’ll connect my repo search function to a Gemini LLM.  
- I’ll give the model a clear system prompt so it knows how to behave.  
- The agent will always search before answering and keep answers specific to my repo.  
- If the answer isn’t found, it will just say “Not found in repo” instead of guessing.  

Basically, today’s step turns my chatbot into a real **project-aware agent** that can reason with repo context.  


In [ ]:
# 🔑 Gemini API setup
import os, google.generativeai as genai

# Enter your API key each run
os.environ["GEMINI_API_KEY"] = input("🔑 Enter Gemini API key: ").strip()
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Pick a Gemini model you have access to
llm = genai.GenerativeModel("gemini-pro-latest")

# Test the connection
resp = llm.generate_content("Hello Gemini, testing Day 4 agent setup.")
print(resp.text)


In [60]:
# Reuse your hybrid_search from Day 3
def dermascan_search(query: str, topk: int = 5):
    """
    Perform a hybrid search on the DermaScan repo.
    Returns top-k chunks for context.
    """
    results = hybrid_search(query, topk=topk)
    chunks = []
    for r in results:
        chunks.append(f"[FILE: {r.get('filename','')}] {r.get('chunk','')}")
    return "\n\n---\n\n".join(chunks)


In [61]:
system_prompt = """
You are a helpful assistant for the DermaScan Android App project.

- Always use the search tool before answering.
- If the repo has the answer, reply with it and mention the file(s).
- If not found, respond with: "Not found in repo."
- Keep answers concise and accurate.
"""
print(system_prompt)



You are a helpful assistant for the DermaScan Android App project.

- Always use the search tool before answering.
- If the repo has the answer, reply with it and mention the file(s).
- If not found, respond with: "Not found in repo."
- Keep answers concise and accurate.



In [62]:
def ask_with_repo(question: str, topk: int = 5):
    # Gather repo context
    context = dermascan_search(question, topk=topk)

    # Build prompt
    prompt = f"""
{system_prompt}

# Repo context:
{context}

# Question:
{question}
"""
    resp = llm.generate_content(prompt)
    return resp.text

# Example Q&A
q = "What dataset is used in this project and what models are applied?"
print("Q:", q)
print("A:", ask_with_repo(q))


Q: What dataset is used in this project and what models are applied?
A: Based on the `README.md` file:

*   **Dataset**: The project uses the [HAM10000](https://www.kaggle.com/kmader/skin-cancer-mnist-ham10000) dataset, which contains over 10,000 dermoscopic images.
*   **Models**:
    *   **Multi-Model Ensemble Learning**: Combines multiple models for decision-making.
    *   **Attention U-Net**: Used for lesion segmentation.
    *   The models are in TensorFlow Lite (TFLite) format.


In [63]:
questions = [
    "Where is the user login implemented in the Android app?",
    "How is the DermaScan app deployed?",
    "What technologies are used for image preprocessing?"
]

for q in questions:
    print("Q:", q)
    print("A:", ask_with_repo(q))
    print("="*80)


Q: Where is the user login implemented in the Android app?
A: Not found in repo.
Q: How is the DermaScan app deployed?
A: Based on the `README.md` file, the app is deployed via GitHub Releases.

You can download the latest app source and model files from the "Releases" section of the GitHub repository. The file to download is `SCapp_split.zip`, which you then need to extract.
Q: What technologies are used for image preprocessing?
A: Based on the `README.md` file, the project uses **OpenCV** for image processing.
